<a href="https://colab.research.google.com/github/ShethNidhi06/Twitter_Data/blob/main/NEG_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Positive Synonym Generator. 

In [69]:
import pandas as pd

In [70]:
url="https://raw.githubusercontent.com/ShethNidhi06/Twitter_Data/main/Tweets.csv"
df1=pd.read_csv(url)
df1=df1.dropna()
df1.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


Creating a DataFrame (df_filtered) containing only negative tweets.


In [71]:
df_filtered = df1[df1['sentiment'] =='negative']
df_filtered=df_filtered.reset_index()
  
# Print the new dataframe
print(df_filtered.head())
print('Length of dataframe:', len(df_filtered))

   index      textID                                               text  \
0      1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
1      2  088c60f138                          my boss is bullying me...   
2      3  9642c003ef                     what interview! leave me alone   
3      4  358bd9e861   Sons of ****, why couldn`t they put them on t...   
4     12  74a76f6e0a       My Sharpie is running DANGERously low on ink   

    selected_text sentiment  
0        Sooo SAD  negative  
1     bullying me  negative  
2  leave me alone  negative  
3   Sons of ****,  negative  
4     DANGERously  negative  
Length of dataframe: 7781


Calculating individual sentiment scores using VADER

In [72]:
!pip install vaderSentiment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
from vaderSentiment.vaderSentiment import *
new_words = {
    '****':-2.5,
}

sid_obj = SentimentIntensityAnalyzer()
sid_obj.lexicon.update(new_words)

def sentiment_scores(text):
    sentiments=[]
    sentitext= SentiText(text)
    words_and_emoticons = sentitext.words_and_emoticons
   
    for item in words_and_emoticons:
            valence = 0
            i = words_and_emoticons.index(item)
    
    

            sentiment_dict = sid_obj.sentiment_valence(valence, sentitext, item, i, sentiments)
            

    return sentiment_dict


In [74]:
sentence=df_filtered.text[1]
print(sentence)
print(sentiment_scores(sentence))
sentiment_scores("****")

my boss is bullying me...
[0, 0, 0, -2.9, 0]


[-2.5]

In [75]:
df_filtered['scores']=df_filtered['text'].map(lambda x:list(sentiment_scores(x)))
df_filtered.reset_index()
df_filtered.head()
#df_filtered.insert(loc=0, column='scores', value=sentiment_scores(df_filtered['text']))



,index,textID,text,selected_text,sentiment,scores
0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]"
1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]"
2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]"
3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]"


In [76]:
#converting the list of scores to a dictionary={index:score}
def dict(list):
  dict={}
  for i in range(len(list)):
    dict[i]=list[i]
  return dict
df_filtered['dictionary']=df_filtered['scores'].apply(lambda x: dict(x))
df_filtered.head()

,index,textID,text,selected_text,sentiment,scores,dictionary
0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6..."
1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}"
2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}"
3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ..."
4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ..."


In [77]:
#extracting the most negative word in the sentence by sorting the dictionary and returning its index 
import operator
from collections import OrderedDict
def sort_dict(dict):
  dict2=sorted(dict.items(), key=operator.itemgetter(1))

  sorted_dict = OrderedDict()
  for k, v in dict2:
    sorted_dict[k] = v
  first_pair = next(iter((sorted_dict.items())) )
  for i in range(len(df_filtered)):
    ele=first_pair[i]
    return int(ele)


df_filtered['neg_word']=df_filtered['dictionary'].apply(lambda x:sort_dict(x))
df_filtered=df_filtered.reset_index()
df_filtered.head()


# def minimum(senti):
#   senti.keys.sort()
  
#   return senti[0]

,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4
3,3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",0
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4


In [78]:
#neg=df_filtered['scores'].sort_values()
#print(neg[1][0])

# type(df_filtered['neg_word'])
# df_filtered.neg_word[0][0]

In [79]:
# def find_word(series):
#   for i in range(len(df_filtered)):
#     ele=series[i]
#     return int(ele)
# df_filtered['ind']=df_filtered['neg_word'].apply(lambda x: find_word(x))
# df_filtered.head()
index=df_filtered['neg_word'][2]  
print(index)
print(df_filtered.text[2].split()[index])


4
alone


Most negative word in sentence 

In [84]:
 
#   for i in range(len(df_filtered)):
#     lst=text.split()
#     index=df_filtered['neg_word'][i]
#     word=lst[index-1]
#     return word

# for i in range(len(df_filtered)):
#   df_filtered['word'][i]=df_filtered['text'][i].split()[df_filtered['neg_word'][i]]

def ext_word():
  list=[]
  for i in range(len(df_filtered)):
    lst=df_filtered['text'][i].split()
    index=df_filtered['neg_word'][i]
    list.append(lst[index])
  return list
  
#store the words in a list and assign it to a new column in the dataframe.
word_list=ext_word()
df_filtered['word']=word_list
df_filtered.head()

,level_0,index,textID,text,selected_text,sentiment,scores,dictionary,neg_word,word
0,0,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0, -2.833, 0, 0, -0.6, 0, 0, 0, 0, 0]","{0: 0, 1: -2.833, 2: 0, 3: 0, 4: -0.6, 5: 0, 6...",1,SAD
1,1,2,088c60f138,my boss is bullying me...,bullying me,negative,"[0, 0, 0, -2.9, 0]","{0: 0, 1: 0, 2: 0, 3: -2.9, 4: 0}",3,bullying
2,2,3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0, 0, -0.2, 0, -1.0]","{0: 0, 1: 0, 2: -0.2, 3: 0, 4: -1.0}",4,alone
3,3,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",0,Sons
4,4,12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative,"[0, 0, 0, 0, -2.0, -1.1, 0, 0]","{0: 0, 1: 0, 2: 0, 3: 0, 4: -2.0, 5: -1.1, 6: ...",4,DANGERously


In [83]:
df_filtered.to_csv('file1.csv')